# 0. Example format

In [ ]:
### This dummy dataframe depicts the format of the input data
### The labels are are provided in a one-hot encoded fashion and as such mutually exclusive
import pandas as pd

d = {'Variable_1': [0, 0, 0, 1, 1, 0, 0, 0, 0, 0], 
     'Variable_2': [0, 1, 0, 0, 0, 1, 0, 1, 0, 1],
     'Variable_3': [1, 0, 1, 0, 0, 0, 1, 0, 1, 0],
     'Report': ['free text']*10}
df = pd.DataFrame(data=d)

# 1. Importing Data

In [ ]:
import pandas as pd
import os

# Set directories
source_dir = ### INSERT SOURCE DIRECTORY ###


# Import Excel file
df = pd.read_excel(os.path.join(source_dir,'df_one_hot_encoded.xlsx'))

df = df[['Glioma', 'Meningioma', 'Metastasis', 'Report', 'Patient_number']]
df[['Glioma', 'Meningioma', 'Metastasis', 'Report', 'Patient_number']].head(10)

# 2. Preprocessing

### 2A. Remove uneccesary patterns

In [ ]:
import re

def preprocessor(text):
    
    #remove all text before 'CLINICAL DATA'
    text = re.sub(r'.*CLINICAL DATA', 'CLINICAL DATA', text)
    
    #remove disclaimer and signature'
    text = text.split('These tests were developed and their performance characteristics determined bythe Molecular Diagnostics Laboratory')[0]
    
    # Remove newlines
    text = text.replace(r'\n', ' ')
    
    # Remove date
    date_pattern = r'[0-9]{1,2}[-/][0-9]{1,2}[-/][0-9]{2,}'
    text = re.sub(date_pattern, ' ', text.lower())
    
    # Remove whitespace
    text = ' '.join(text.split())
    
    # Remove punctution, keep decimal points
    text = re.sub(r'[\W]+(?!\d)', ' ', text)

    return text

df['cleaned_report'] = df['Report'].apply(preprocessor)
df['cleaned_report'][1]

### 2B. Remove stop words

In [ ]:
from nltk.corpus import stopwords
import nltk

nltk.download('stopwords')

stop_words_english = stopwords.words('english')

stop_words_modified = [w for w in stop_words_english if w not in ['no', 'not']]

def remove_words(clean_report): 
    clean_report_words = clean_report.split()
    stripped_report_words  = [word for word in clean_report_words if word.lower() not in stop_words_modified]
    stripped_report_text = ' '.join(stripped_report_words)
    return stripped_report_text

df['stripped_report'] = df['cleaned_report'].apply(remove_words)
df.head(2)

### 2C. Stemming

In [ ]:
from nltk.stem.porter import PorterStemmer

porter = PorterStemmer()

def porter_stemming(text):
    stemmed_report_words = [porter.stem(word) for word in text.split()]
    stemmed_report_text = ' '.join(stemmed_report_words)
    return stemmed_report_text

df['stemmed_report'] = df['stripped_report'].apply(porter_stemming)
df.head(2)

### 2D. Tokenizing

In [ ]:
def split_words(report): 
    report = report.split()
    return report

df['tokenized_report'] = df['stemmed_report'].apply(split_words)

df.head(2)

### 2E.  Train validation test split

In [ ]:
import numpy as np
from sklearn.model_selection import train_test_split

list_train, list_validation_test = train_test_split(df.Patient_number.unique(), test_size = 0.5)
list_validation, list_test = train_test_split(list_validation_test, test_size = 0.5)
print('n patients in training set:', len(list_train))
print('n patients in validation set:', len(list_validation))
print('n patients in test set:', len(list_test))

df.loc[df['Patient_number'].isin(list_train), 'cohort'] = "train"
df.loc[df['Patient_number'].isin(list_validation), 'cohort'] = "validation"
df.loc[df['Patient_number'].isin(list_test), 'cohort'] = "test"
df.head(2)

### 2F. Save Reports

In [ ]:
target_dir = ### INSERT TARGET DIRECTORY ###
df.to_excel(os.path.join(target_dir, 'preprocessed_reports.xlsx')) 

### 2H. Load Preprocessed Reports 

In [ ]:
import pandas as pd
import os

source_dir = ### INSERT SOURCE DIRECTORY ###
df = pd.read_excel(os.path.join(source_dir, 'preprocessed_reports.xlsx')) 
print('Dimensions df are', df.shape)
df.head(5)

# 3 Statistical / Classic Machine Learning

### 3A. Create an parameter grid

In [ ]:
from sklearn.model_selection import ParameterGrid

param_grid_dict = {'diagnosis': ['Glioma', 'Meningioma', 'Metastasis'],
                    'l1': [1, 2, 4, 8, 16, 32],
                   'max_features':[100, 250, 500, 1000, 1500, 2000],
                   'ngram_range': [1, 2, 3],
                   'sample_size': [25, 50, 75, 100, 150, 200, 250, 300, 400, 500, 600, 800, 1000, 1200, 1500, 1800, 2100, 2500, 3000] 
                  }

param_grid_list = list(ParameterGrid(param_grid_dict))
param_grid_list

parameter_grid = []
for dict in param_grid_list:
    hyperparameters = list(dict.values())
    parameter_grid.append(hyperparameters)

print('Length parameter grid is', len(parameter_grid), 'hyperparameter settings')
parameter_grid

### 3D. Hyperparameter tuning regression-based models

In [ ]:
### Running this code block on a multiple cores or a cloud service might be desirable

from sklearn.model_selection import cross_val_score, GridSearchCV, StratifiedKFold, train_test_split, KFold
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression, Lasso 
from sklearn.metrics import roc_auc_score
from sklearn import linear_model
import scipy.sparse as sp
import pandas as pd
import numpy as np
import math
import os

def square(list):
    return [i ** 2 for i in list]

diagnosis_column = []
l1_column = []
max_features_column = []
ngram_range_column = []
sample_size_column = []
auc_mean_column = []
auc_std_column = []
number_of_bootstraps = []

start_gridsearch = 1

for diagnosis, l1, max_features, ngram_range, sample_size in parameter_grid:
    
    print('##### Grid Search', start_gridsearch,'/', len(parameter_grid))
    
    word_vectorizer = TfidfVectorizer(ngram_range=(1,ngram_range),max_features=max_features)
    word_vectorizer.fit(df['tokenized_report'][df['cohort'].isin(['train', 'validation'])])
  
    bootstrap_aucs = []
    
    start_bootstrap = 1
    
    for n in range(int(3000//sample_size)):
        
        df_frac = df[(df['cohort'] == 'train')].sample(n = sample_size, replace = True)
    
        X_train = word_vectorizer.transform(df_frac['tokenized_report']).toarray()
        y_train = np.asarray(df_frac[diagnosis])
        
        X_val = word_vectorizer.transform(df['tokenized_report'][df['cohort'].isin(['validation'])]).toarray()
        y_val = np.asarray(df[diagnosis][df['cohort'].isin(['validation'])])
        
        try:
            model_lasso = LogisticRegression(penalty = 'l1', C=l1, max_iter=200)
            model_lasso.fit(X_train, y_train)

            bootstrap_auc = roc_auc_score(y_val, model_lasso.predict_proba(X_val)[:,1])
            bootstrap_aucs.append(bootstrap_auc)
        
        except ValueError:
               pass
                                            
        start_bootstrap = start_bootstrap + 1 
                                          
    n_bootstraps = start_bootstrap - 1
    
    print('Number of bootstraps is', n_bootstraps)                                     
                                          
    auc_mean = np.mean(bootstrap_aucs)
    auc_std = np.std(bootstrap_aucs)

    print('The mean bootstrapped AUC for grid search', start_gridsearch, 'bootstrap ', start_bootstrap,
          round(auc_mean,2),'±', round(auc_std,3))
    
    diagnosis_column.append(diagnosis)
    l1_column.append(l1)
    max_features_column.append(max_features)
    ngram_range_column.append(ngram_range)
    sample_size_column.append(sample_size)
    auc_mean_column.append(auc_mean)
    auc_std_column.append(auc_std)
    number_of_bootstraps.append(n_bootstraps)
    
    start_gridsearch = start_gridsearch + 1
    
df_performance = pd.DataFrame(list(zip(diagnosis_column, sample_size_column, l1_column, max_features_column, ngram_range_column,
                                    auc_mean_column, auc_std_column, number_of_bootstraps)))

df_performance.columns = ['diagnosis', 'sample_size', 'l1', 'max_features', 'ngram_range',  'auc_mean', 
                          'auc_std', 'number_of_bootstraps']

target_dir = ### INSERT TARGET DIRECTORY ###
df_performance.to_excel(os.path.join(target_dir, 'hyperparameter_tuning.xlsx'))

df_performance

### 3D. Select optimal hyperparameter settings

In [ ]:
import pandas as pd
import os
import numpy as np

target_dir = ### INSERT SOURCE DIRECTORY ###
df_hyperparameter_results = pd.read_excel(os.path.join(target_dir, 'iteration_1.xlsx'))

df_hyperparameter_optimal_lasso = df_hyperparameter_results.loc[df_hyperparameter_results.reset_index().groupby(['diagnosis', 'sample_size'])['auc_mean'].idxmax()]
df_hyperparameter_optimal_lasso = df_hyperparameter_optimal_lasso[['diagnosis', 'l1', 'max_features', 'ngram_range', 'sample_size']]
optimal_paramater_grid_lasso = df_hyperparameter_optimal_lasso.values.tolist()
optimal_paramater_grid_lasso

df_hyperparameter_results_logistic = df_hyperparameter_results.loc[df_hyperparameter_results['l1'] == 1]
df_hyperparameter_optimal_logistic = df_hyperparameter_results_logistic.loc[df_hyperparameter_results_logistic.groupby(['diagnosis', 'sample_size'])['auc_mean'].idxmax()]
df_hyperparameter_optimal_logistic = df_hyperparameter_optimal_logistic[['diagnosis', 'l1', 'max_features', 'ngram_range', 'sample_size']]
optimal_paramater_grid_logistic = df_hyperparameter_optimal_logistic.values.tolist()
optimal_paramater_grid_logistic

### 3E. Train final models and compute predicted probabilities and classses

In [ ]:
### Run this code block twice. Once for Lasso, once for logistic regression.

from sklearn.model_selection import cross_val_score, GridSearchCV, StratifiedKFold, train_test_split, KFold
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression, Lasso 
from sklearn.metrics import roc_auc_score
from keras.utils import to_categorical
from sklearn import linear_model
import scipy.sparse as sp
import pandas as pd
import numpy as np
import os

def square(list):
    return [i ** 2 for i in list]

diagnosis_column = []
l1_column = []
max_features_column = []
ngram_range_column = []
sample_size_column = []
auc_mean = []
auc_std = []
number_of_bootstraps = []

gridsearch = 1

for diagnosis, l1, max_features, ngram_range, sample_size in optimal_paramater_grid_logistic:
    
    print('##### Grid Search', gridsearch,'/', len(optimal_paramater_grid_lasso))
    
    word_vectorizer = TfidfVectorizer(ngram_range=(1,ngram_range),max_features=max_features)
    word_vectorizer.fit(df['tokenized_report'][df['cohort'].isin(['train', 'test'])])

    bootstrap_aucs = [] 
    
    start_bootstrap = 1
    
    for n in range(100):
        
        df_frac = df[(df['cohort'] == 'train')].sample(n = sample_size, replace = True)
    
        X_train = word_vectorizer.transform(df_frac['tokenized_report']).toarray()
        y_train = np.asarray(df_frac[diagnosis])
        
        X_test = word_vectorizer.transform(df['tokenized_report'][df['cohort'].isin(['test'])]).toarray()
        y_test = np.asarray(df[diagnosis][df['cohort'].isin(['test'])])
        
        try:
            model = LogisticRegression(penalty = 'l1', C=l1, max_iter=200)
            model.fit(X_train, y_train)

            predicted_probabilities = model.predict_proba(X_test)[:,1]
            auc = roc_auc_score(y_test, predicted_probabilities)

        except ValueError:
              pass
        
        print('AUC:', str(round(auc,3)))
        bootstrap_aucs.append(auc)
        
        gridsearch += 1 
                                          
    n_bootstraps = gridsearch - 1
    print('### Number of bootstraps is', n_bootstraps)
    
    bootstrap_auc_mean = np.mean(bootstrap_aucs)
    bootstrap_auc_std = np.std(bootstrap_aucs)
    print('### Mean AUC for grid search', gridsearch, ':', round(bootstrap_auc_mean,2),'±', round(bootstrap_auc_std,3))

    print('')
    
    diagnosis_column.append(diagnosis)
    l1_column.append(l1)
    max_features_column.append(max_features)
    ngram_range_column.append(ngram_range)
    sample_size_column.append(sample_size)
    auc_mean.append(bootstrap_auc_mean)
    auc_std.append(bootstrap_auc_std)
    number_of_bootstraps.append(n_bootstraps)
    
    start_gridsearch = start_gridsearch + 1
    
df_performance = pd.DataFrame(list(zip(diagnosis_column, sample_size_column, l1_column, max_features_column, ngram_range_column,
                                    auc_mean, auc_std, number_of_bootstraps)))

df_performance.columns = ['diagnosis', 'sample_size', 'l1', 'max_features', 'ngram_range',  'auc_mean', 
                          'auc_std', 'number_of_bootstraps']

target_dir = ### INSERT SOURCE DIRECTORY ###

df_performance.to_excel(os.path.join(target_dir, 'final_results_logistic.xlsx'))

df_performance
                            

### 3F. Get results lasso regression model

In [4]:
import pandas as pd
import os

path_final_results = ### INSERT SOURCE DIRECTORY ###
df_results_lasso = pd.read_excel(os.path.join(path_final_results, 'pooled_final_results_lasso.xlsx'))
df_results_lasso

,sample_size,pooled_auc,pooled_auc_std
0,25,0.697872,0.105783
1,50,0.850333,0.069648
2,75,0.904126,0.033465
3,100,0.929045,0.022178
4,150,0.949481,0.014136
5,200,0.958935,0.010419
6,250,0.964847,0.008759
7,300,0.968405,0.007763
8,400,0.973312,0.005666
9,500,0.977467,0.003977


### 3G. Get results logistic regression model

In [5]:
import pandas as pd
import os

path_final_results = ### INSERT SOURCE DIRECTORY ###
df_results_logistic = pd.read_excel(os.path.join(path_final_results, 'pooled_final_results_logistic.xlsx'))
df_results_logistic

,sample_size,pooled_auc,pooled_auc_std
0,25,0.506660,0.028843
1,50,0.582606,0.108954
2,75,0.715015,0.095543
3,100,0.785471,0.078212
4,150,0.890234,0.034359
5,200,0.918495,0.019208
6,250,0.928700,0.014242
7,300,0.943737,0.010542
8,400,0.950385,0.008844
9,500,0.961471,0.008590


# 4. Deep Learning

### 4A. Load preprocessed reports

In [ ]:
import pandas as pd
import os

source_dir = ### INSERT SOURCE DIRECTORY ###
df = pd.read_excel(os.path.join(source_dir, 'preprocessed_reports.xlsx')) 
print('Dimensions df are', df.shape)
df.head(10)

### 4B. Create a parameter grid deep learning model

In [ ]:
# In total, we tested 58 settings across 15 hyperparameters resulting in a hyperparameter space 
# that encompasses 1,300,561,920 hypothetical model architectures.
# Because it is virtually impossible to evaluate all these model architectures from a computational standpoint, 
# we applied an hierarchical approach for hyperparameter optimization by subsequently optimizing the:
# 1) preprocessing settings (embedding_dim, max_len, max_words)
# 2) network architecture (double_layer, filter_size, n_filters_conv, n_filters_dense, n_layers_conv, n_layers_dense)
# 3) regularizers (dropout, l1, l2, max_pooling)
# 4) learning speed (optimizer_type, learning_rate)

from sklearn.model_selection import ParameterGrid

param_grid_dict = { 
    'double_layer': [0,1], 
    'dropout': [0, 0.1, 0.2, 0.5], 
    'embedding_dim': [8, 16, 32, 64, 128, 256, 512],
    'filter_size': [1,3,5,7,9], 
    'l1': [0.00000, 0.00001, 0.0001, 0.001],
    'l2': [0.00000, 0.00001, 0.0001, 0.001], 
    'learning_rate': [0.002, 0.005, 0.01, 0.05], 
    'max_len': [250, 500, 1000, 2000],
    'max_pooling' : [0,1],
    'max_words': [250, 500, 750, 1000, 1500, 2000, 5000],
    'n_filters_conv:': [8, 16, 32, 64],
    'n_filters_dense':[8,16,32],
    'n_layers_conv': [0,1,2,3],
    'n_layers_dense':[0,1],
    'optimizer_type': ['adam', 'nadam'],
    'sample_size': [25, 50, 100, 150, 200, 250, 300, 400, 500, 750, 1000, 1500, 2000, 3000],
                  }

param_grid_list = list(ParameterGrid(param_grid_dict))

parameter_grid = []
for dict in param_grid_list:
    hyperparameters = list(dict.values())
    parameter_grid.append(hyperparameters)

print('Length parameter grid is', len(parameter_grid), 'hyperparameter settings')

parameter_grid

###  4C. Hyperparameter tuning deep learning

In [ ]:
### In this code block, two functions were used to automatically construct and evaluate different model architectures.
### Running this code block on a multiple cores or a cloud service might be desirable.
### Due to the hierarchical approach of the hyperparameter optimization,
### the best model architecture is not automatically extracted 
### but retrieved through and iterative proces.

from keras.layers import Embedding, Flatten, Dense, Input, LSTM, Conv1D, Dropout, SpatialDropout1D
from sklearn.model_selection import train_test_split, KFold
from keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from keras.preprocessing.text import Tokenizer
from sklearn.metrics import roc_auc_score
from keras.callbacks import EarlyStopping
from keras.models import Sequential
from keras import preprocessing
from keras import regularizers
from keras import optimizers
from keras import layers
from numpy import array
import pandas as pd
import numpy as np
import math
import time

bootstrap_time_column = []
n_layers_dense_column = []
double_layer_column = []
dropout_column = []
embedding_dim_column = []
filter_size_column = []
l1_column = []
l2_column = []
learning_rate_column = []
max_len_column = []
max_pooling_column = []
max_words_column = []
n_filters_dense_column = []
n_filters_conv_column = []
n_layers_conv_column = []
n_params_column = []
optimizer_type_column = []
sample_size_column = []
auc_mean = []
auc_std = []
epochmeans = []
epochstds = []

gridsearch = 1


def get_conv1d_model(n_layers_conv, double_layer, n_filters_conv, filter_size, l1, l2, dropout):
    model = Sequential()
    model.add(Embedding(max_words, embedding_dim, input_length=max_len))
    if n_layers_conv == 0:
        model.add(layers.MaxPooling1D(filter_size))
    elif n_layers_conv == 1:
        if double_layer == 0:
            model.add(layers.Conv1D(n_filters_conv, filter_size, activation='relu', kernel_regularizer=regularizers.l1_l2(l1, l2)))
        elif double_layer == 1:
            model.add(layers.Conv1D(n_filters_conv, filter_size, activation='relu', kernel_regularizer=regularizers.l1_l2(l1, l2)))
            model.add(layers.Conv1D(n_filters_conv, filter_size, activation='relu', kernel_regularizer=regularizers.l1_l2(l1, l2)))
        model.add(layers.MaxPooling1D(filter_size))
    elif n_layers_conv == 2:
        if double_layer == 0:
            model.add(layers.Conv1D(n_filters_conv, filter_size, activation='relu', kernel_regularizer=regularizers.l1_l2(l1, l2)))
        elif double_layer == 1:
            model.add(layers.Conv1D(n_filters_conv, filter_size, activation='relu', kernel_regularizer=regularizers.l1_l2(l1, l2)))
            model.add(layers.Conv1D(n_filters_conv, filter_size, activation='relu', kernel_regularizer=regularizers.l1_l2(l1, l2)))
        model.add(layers.MaxPooling1D(filter_size))
        if double_layer == 0:
            model.add(layers.Conv1D(n_filters_conv*2, filter_size, activation='relu', kernel_regularizer=regularizers.l1_l2(l1, l2)))
        elif double_layer == 1:
            model.add(layers.Conv1D(n_filters_conv*2, filter_size, activation='relu', kernel_regularizer=regularizers.l1_l2(l1, l2)))
            model.add(layers.Conv1D(n_filters_conv*2, filter_size, activation='relu', kernel_regularizer=regularizers.l1_l2(l1, l2)))            
        model.add(layers.MaxPooling1D(filter_size))
    elif n_layers_conv == 3:
        if double_layer == 0:
            model.add(layers.Conv1D(n_filters_conv, filter_size, activation='relu', kernel_regularizer=regularizers.l1_l2(l1, l2)))
        elif double_layer == 1:
            model.add(layers.Conv1D(n_filters_conv, filter_size, activation='relu', kernel_regularizer=regularizers.l1_l2(l1, l2)))
            model.add(layers.Conv1D(n_filters_conv, filter_size, activation='relu', kernel_regularizer=regularizers.l1_l2(l1, l2)))
        model.add(layers.MaxPooling1D(filter_size))
        if double_layer == 0:
            model.add(layers.Conv1D(n_filters_conv*2, filter_size, activation='relu', kernel_regularizer=regularizers.l1_l2(l1, l2)))
        elif double_layer == 1:
            model.add(layers.Conv1D(n_filters_conv*2, filter_size, activation='relu', kernel_regularizer=regularizers.l1_l2(l1, l2)))
            model.add(layers.Conv1D(n_filters_conv*2, filter_size, activation='relu', kernel_regularizer=regularizers.l1_l2(l1, l2)))             
        model.add(layers.MaxPooling1D(filter_size))
        if double_layer == 0:
            model.add(layers.Conv1D(n_filters_conv*4, filter_size, activation='relu', kernel_regularizer=regularizers.l1_l2(l1, l2)))
        elif double_layer == 1:
            model.add(layers.Conv1D(n_filters_conv*4, filter_size, activation='relu', kernel_regularizer=regularizers.l1_l2(l1, l2)))
            model.add(layers.Conv1D(n_filters_conv*4, filter_size, activation='relu', kernel_regularizer=regularizers.l1_l2(l1, l2)))
        model.add(layers.MaxPooling1D(filter_size))
    model.add(layers.GlobalMaxPooling1D())
    model.add(Dropout(dropout))
    model.add(layers.Dense(3, activation='softmax'))
    model.compile(optimizer=nadam, loss='categorical_crossentropy', metrics=['acc'])
    return model

def get_conv1d_model2(n_layers_conv, n_filters_conv, filter_size, l1, l2, max_pooling, n_layers_dense, n_filters_dense, dropout, final_optimizer):
    model = Sequential()
    model.add(Embedding(max_words, embedding_dim, input_length=max_len))
    model.add(SpatialDropout1D(dropout))
    if n_layers_conv == 1:
        model.add(layers.Conv1D(n_filters_conv, filter_size, activation='relu', kernel_regularizer=regularizers.l1_l2(l1, l2)))
    if max_pooling == 1:
        model.add(layers.MaxPooling1D(filter_size))
    model.add(layers.GlobalMaxPooling1D())
    if n_layers_dense == 1:
        model.add(Dense(n_filters_dense, activation='relu'))
    model.add(Dropout(dropout))
    model.add(layers.Dense(3, activation='softmax'))
    model.compile(optimizer=final_optimizer, loss='categorical_crossentropy', metrics=['acc'])
    return model

def square(list):
    return [i ** 2 for i in list]

df_train_val = df[df['cohort'].isin(['train', 'validation'])]

for double_layer, dropout, embedding_dim, filter_size, l1, l2, learning_rate, max_len, max_pooling, max_words, n_filters_conv, n_filters_dense, n_layers_conv, n_layers_dense, optimizer_type, sample_size in parameter_grid:
    
    print('##### Grid Search', gridsearch,'/', len(parameter_grid))
    
    tokenizer = Tokenizer(num_words=max_words)
    tokenizer.fit_on_texts(df_train_val['stemmed_report'])
    sequences = tokenizer.texts_to_sequences(df_train_val['stemmed_report'])
    
    df_train_val['vectorized_report'] = sequences 
    
    bootstrap_aucs = []
    bootstrap_auc_stds = []
    
    bootstrap_epoch = []
    bootstrap_epoch_std = []
    
    bootstrap = 1

    if optimizer_type == 'adam':
        final_optimizer = optimizers.Adam(lr=learning_rate)
    elif optimizer_type == 'nadam':
        final_optimizer = optimizers.Nadam(lr=learning_rate)
    
    #model = get_conv1d_model(n_layers_conv, double_layer, n_filters_conv, filter_size, l1, l2, dropout)
    model = get_conv1d_model2(n_layers_conv, n_filters_conv, filter_size, l1, l2, max_pooling, n_layers_dense, n_filters_dense, dropout, final_optimizer)
    model.summary()

    start_time = time.time()
          
    for n in range(int(3000//sample_size)): 
        
        print('--- Bootstrap', bootstrap)

        df_frac = df_train_val[(df_train_val['cohort'] == 'train')].sample(n = 2*sample_size, replace = True)

        array_train = array(list(df_frac['vectorized_report'][(df_frac['cohort'] == 'train')]))
        array_val = array(list(df_train_val['vectorized_report'][(df_train_val['cohort'] == 'validation')]))
        X_train = preprocessing.sequence.pad_sequences(array_train, maxlen=max_len)
        X_val = preprocessing.sequence.pad_sequences(array_val, maxlen=max_len) 
        
        y_train = array(df_frac[['Glioma', 'Meningioma', 'Metastasis']][(df_frac['cohort'] == 'train')])
        y_val = array(df_train_val[['Glioma', 'Meningioma', 'Metastasis']][(df_train_val['cohort'] == 'validation')])
        
        #model = get_conv1d_model(n_layers_conv, double_layer, n_filters_conv, filter_size, l1, l2, dropout)
        model = get_conv1d_model2(n_layers_conv, n_filters_conv, filter_size, l1, l2, max_pooling, n_layers_dense, n_filters_dense, dropout, final_optimizer)
        
        el_cb = EarlyStopping(patience=20)
        
        try:
            history = model.fit(X_train, y_train,
                epochs=5000,
                batch_size=32,
                validation_split=0.5,
                callbacks=[el_cb],
                verbose = 0)
            
            auc = roc_auc_score(y_val, model.predict_proba(X_val))
            n_epochs = len(history.epoch)
        except ValueError:
                pass
        
        print('AUC:', str(round(auc,3)))
        bootstrap_aucs.append(auc)

        print('n epochs:', str(round(n_epochs,3)))
        bootstrap_epoch.append(n_epochs)
        
        bootstrap += 1

    time_5_bootstraps = time.time() - start_time
    
    n_bootstraps = bootstrap - 1
    
    print('### Number of bootstraps is', n_bootstraps)
    
    bootstrap_auc_mean = np.mean(bootstrap_aucs)
    bootstrap_auc_std = np.std(bootstrap_aucs)
    
    print('### Mean AUC for grid search', gridsearch, ':', round(bootstrap_auc_mean,2),'±', round(bootstrap_auc_std,3))
    
    bootstrap_epoch_mean = np.mean(bootstrap_epoch)
    bootstrap_epoch_std = np.std(bootstrap_epoch)
    
    print('### Mean n epochs for grid search', gridsearch, ':', round(bootstrap_epoch_mean,2),'±', round(bootstrap_epoch_std,3))
    
    print('')
    
    bootstrap_time_column.append(time_5_bootstraps)
    double_layer_column.append(double_layer)
    dropout_column.append(dropout)
    embedding_dim_column.append(embedding_dim)
    filter_size_column.append(filter_size)
    l1_column.append(l1)
    l2_column.append(l2)
    learning_rate_column.append(learning_rate)
    max_len_column.append(max_len)
    max_pooling_column.append(max_pooling)
    max_words_column.append(max_words)
    n_filters_dense_column.append(n_filters_dense)
    n_filters_conv_column.append(n_filters_conv)
    n_layers_conv_column.append(n_layers_conv)
    n_layers_dense_column.append(n_layers_dense)
    n_params_column.append(model.count_params())
    optimizer_type_column.append(optimizer_type)
    sample_size_column.append(sample_size)
    auc_mean.append(bootstrap_auc_mean)
    auc_std.append(bootstrap_auc_std)
    epochmeans.append(bootstrap_epoch_mean)
    epochstds.append(bootstrap_epoch_std)
    
    gridsearch += 1

    
df_performance = pd.DataFrame(list(zip(sample_size_column, embedding_dim_column, max_len_column, max_words_column, 
                                       n_layers_conv_column, double_layer_column, n_filters_conv_column, filter_size_column,
                                       max_pooling_column, n_layers_dense_column, n_filters_dense_column,
                                       l1_column, l2_column, dropout_column, n_params_column,
                                       auc_mean, auc_std, epochmeans, epochstds, optimizer_type_column, learning_rate_column, bootstrap_time_column)))


df_performance.columns = ['sample_size', 'embedding_dim', 'max_len', 'max_words',
                         'n_layers_conv', 'double_layer', 'n_filters_conv', 'filter_size',
                         'max_pooling', 'n_layers_dense', 'n_filters_dense',
                         'l1_column', 'l2_column', 'dropout', 'n_params',
                         'auc_mean', 'auc_std', 'epochmeans', 'epochstds',
                         'optimizer_type_column', 'learning_rate_column', 'bootstrap_time_column']

target_dir = ### INSERT TARGET DIRECTORY ###

df_performance.to_excel(os.path.join(target_dir, 'hyperparam_tuning_cnn.xlsx'))

df_performance

### 4D. Create optimal parametergrid

In [ ]:
### Because the bootstrapped predictions on the final test required significant computational time,
### the results were computed in 4 batches based on the sample size and merged together.

from sklearn.model_selection import ParameterGrid

param_grid_dict = { 
    'double_layer': [0], #<- final
    'dropout': [0.5], #<- final
    'embedding_dim': [128],# <- final
    'filter_size': [3], #<- final 
    'l1': [0],#<- final
    'l2': [0], #<- final
    'learning_rate': [0.01], #<- final
    'max_len': [1000],#<- final
    'max_pooling' : [1],#<- final
    'max_words': [1000],#<- final
    'n_filters_conv:': [0],#<- final
    'n_filters_dense':[0],#<- final
    'n_layers_conv': [0],#<- final
    'n_layers_dense':[0],#<- final
    'optimizer_type': ['adam'],
    'sample_size': [25, 50, 100, 150, 200, 250, 300, 400, 500, 750, 1000, 1500, 2000, 3000],
                  }

param_grid_list = list(ParameterGrid(param_grid_dict))

parameter_grid = []
for dict in param_grid_list:
    hyperparameters = list(dict.values())
    parameter_grid.append(hyperparameters)


print('Length parameter grid is', len(parameter_grid), 'hyperparameter settings')

parameter_grid

### 4E. Create final performance metrics

In [ ]:
# Both the CNN and non-CNN model architecture can be evaluated by specifying the get_conv1d_model2 function

from keras.layers import Embedding, Flatten, Dense, Input, LSTM, Conv1D, Dropout, SpatialDropout1D
from sklearn.model_selection import train_test_split, KFold
from keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from keras.preprocessing.text import Tokenizer
from sklearn.metrics import roc_auc_score
from keras.callbacks import EarlyStopping
from keras.models import Sequential
from keras import preprocessing
from keras import regularizers
from keras import optimizers
from keras import layers
from numpy import array
import pandas as pd
import numpy as np
import math
import time

bootstrap_time_column = []
n_layers_dense_column = []
double_layer_column = []
dropout_column = []
embedding_dim_column = []
filter_size_column = []
l1_column = []
l2_column = []
learning_rate_column = []
max_len_column = []
max_pooling_column = []
max_words_column = []
n_filters_dense_column = []
n_filters_conv_column = []
n_layers_conv_column = []
n_params_column = []
optimizer_type_column = []
sample_size_column = []
auc_mean = []
auc_std = []
epochmeans = []
epochstds = []

gridsearch = 1

def get_conv1d_model2(n_layers_conv, n_filters_conv, filter_size, l1, l2, max_pooling, n_layers_dense, n_filters_dense, dropout, final_optimizer):
    model = Sequential()
    model.add(Embedding(max_words, embedding_dim, input_length=max_len))
    model.add(SpatialDropout1D(dropout))
    if n_layers_conv == 1:
        model.add(layers.Conv1D(n_filters_conv, filter_size, activation='relu', kernel_regularizer=regularizers.l1_l2(l1, l2)))
    if max_pooling == 1:
        model.add(layers.MaxPooling1D(filter_size))
    model.add(layers.GlobalMaxPooling1D())
    if n_layers_dense == 1:
        model.add(Dense(n_filters_dense, activation='relu'))
    model.add(Dropout(dropout))
    model.add(layers.Dense(3, activation='softmax'))
    model.compile(optimizer=final_optimizer, loss='categorical_crossentropy', metrics=['acc'])
    return model


def square(list):
    return [i ** 2 for i in list]

for double_layer, dropout, embedding_dim, filter_size, l1, l2, learning_rate, max_len, max_pooling, max_words, n_filters_conv, n_filters_dense, n_layers_conv, n_layers_dense, optimizer_type, sample_size in parameter_grid:
    
    print('##### Grid Search', gridsearch,'/', len(parameter_grid))
    
    tokenizer = Tokenizer(num_words=max_words)
    tokenizer.fit_on_texts(df['stemmed_report'])
    sequences = tokenizer.texts_to_sequences(df['stemmed_report'])
    
    df['vectorized_report'] = sequences 
    
    bootstrap_aucs = []
    bootstrap_auc_stds = []
    
    bootstrap_epoch = []
    
    bootstrap = 1

    if optimizer_type == 'adam':
        final_optimizer = optimizers.Adam(lr=learning_rate)
    elif optimizer_type == 'nadam':
        final_optimizer = optimizers.Nadam(lr=learning_rate)
    
    model = get_conv1d_model2(n_layers_conv, n_filters_conv, filter_size, l1, l2, max_pooling, n_layers_dense, n_filters_dense, dropout, final_optimizer)
    model.summary()

    start_time = time.time()
          
    for n in range(100):
        
        print('--- Bootstrap', bootstrap)

        df_frac = df[(df['cohort'] == 'train')].sample(n = 2*sample_size, replace = True)

        array_train = array(list(df_frac['vectorized_report'][(df_frac['cohort'] == 'train')]))
        array_test = array(list(df['vectorized_report'][(df['cohort'] == 'test')]))

        X_train = preprocessing.sequence.pad_sequences(array_train, maxlen=max_len)
        X_test = preprocessing.sequence.pad_sequences(array_test, maxlen=max_len)
        
        y_train = array(df_frac[['Glioma', 'Meningioma', 'Metastasis']][(df_frac['cohort'] == 'train')])
        y_test = array(df[['Glioma', 'Meningioma', 'Metastasis']][(df['cohort'] == 'test')])
        
        model = get_conv1d_model2(n_layers_conv, n_filters_conv, filter_size, l1, l2, max_pooling, n_layers_dense, n_filters_dense, dropout, final_optimizer)
        
        try:
            history = model.fit(X_train, y_train,
                epochs=50,
                batch_size=32,
                verbose = 0)
            
            auc = roc_auc_score(y_test, model.predict_proba(X_test), average='weighted')
            n_epochs = len(history.epoch)
        except ValueError:
                pass
        
        print('AUC:', str(round(auc,3)))
        bootstrap_aucs.append(auc)

        print('n epochs:', str(round(n_epochs,3)))
        bootstrap_epoch.append(n_epochs)
        
        bootstrap += 1

    time_100_bootstraps = time.time() - start_time
    
    n_bootstraps = bootstrap - 1
    print('### Number of bootstraps is', n_bootstraps)
    
    bootstrap_auc_mean = np.mean(bootstrap_aucs)
    bootstrap_auc_std = np.std(bootstrap_aucs)
    print('### Mean AUC for grid search', gridsearch, ':', round(bootstrap_auc_mean,2),'±', round(bootstrap_auc_std,3))

    bootstrap_epoch_mean = np.mean(bootstrap_epoch)
    bootstrap_epoch_std = np.std(bootstrap_epoch)
    print('### Mean n epochs for grid search', gridsearch, ':', round(bootstrap_epoch_mean,2),'±', round(bootstrap_epoch_std,3))
    
    print('')
    
    bootstrap_time_column.append(time_100_bootstraps)
    double_layer_column.append(double_layer)
    dropout_column.append(dropout)
    embedding_dim_column.append(embedding_dim)
    filter_size_column.append(filter_size)
    l1_column.append(l1)
    l2_column.append(l2)
    learning_rate_column.append(learning_rate)
    max_len_column.append(max_len)
    max_pooling_column.append(max_pooling)
    max_words_column.append(max_words)
    n_filters_dense_column.append(n_filters_dense)
    n_filters_conv_column.append(n_filters_conv)
    n_layers_conv_column.append(n_layers_conv)
    n_layers_dense_column.append(n_layers_dense)
    n_params_column.append(model.count_params())
    optimizer_type_column.append(optimizer_type)
    sample_size_column.append(sample_size)
    auc_mean.append(bootstrap_auc_mean)
    auc_std.append(bootstrap_auc_std)
    epochmeans.append(bootstrap_epoch_mean)
    epochstds.append(bootstrap_epoch_std)
    
    gridsearch += 1

    
df_performance = pd.DataFrame(list(zip(sample_size_column, embedding_dim_column, max_len_column, max_words_column, 
                                       n_layers_conv_column, double_layer_column, n_filters_conv_column, filter_size_column,
                                       max_pooling_column, n_layers_dense_column, n_filters_dense_column,
                                       l1_column, l2_column, dropout_column, n_params_column,
                                       auc_mean, auc_std,
                                       epochmeans, epochstds, optimizer_type_column, learning_rate_column, bootstrap_time_column)))


df_performance.columns = ['sample_size', 'embedding_dim', 'max_len', 'max_words',
                         'n_layers_conv', 'double_layer', 'n_filters_conv', 'filter_size',
                         'max_pooling', 'n_layers_dense', 'n_filters_dense',
                         'l1_column', 'l2_column', 'dropout', 'n_params',
                         'auc_mean', 'auc_std', 'epochmeans', 'epochstds',
                         'optimizer_type_column', 'learning_rate_column', 'bootstrap_time_column']

target_dir = ### INSERT TARGET DIRECTORY ###

df_performance.to_excel(os.path.join(target_dir, 'hyperparam_tuning_cnn.xlsx'))

df_performance

### 4F. Merge final results of the classic CNN model 

In [10]:
### Because the bootstrapped predictions on the final test required significant computational time,
### the results were computed in 4 batches based on the sample size and merged together.


import pandas as pd
import os

path_final_results = ### INSERT TARGET DIRECTORY ###
df_results_part_1 = pd.read_excel(os.path.join(path_final_results, 'part_1.xlsx'))
df_results_part_2 = pd.read_excel(os.path.join(path_final_results, 'part_2.xlsx'))
df_results_part_3 = pd.read_excel(os.path.join(path_final_results, 'part_3.xlsx'))
df_results_part_4 = pd.read_excel(os.path.join(path_final_results, 'part_4.xlsx'))

df_results_cnn = df_results_part_1.append(df_results_part_2, ignore_index=True)
df_results_cnn = df_results_cnn.append(df_results_part_3, ignore_index=True)
df_results_cnn = df_results_cnn.append(df_results_part_4, ignore_index=True)

df_results_cnn = df_results_cnn[['sample_size', 'auc_mean', 'auc_std']]

df_results_cnn.to_excel(os.path.join(path_final_results, 'final_results.xlsx'))
df_results_cnn

,sample_size,auc_mean,auc_std
0,25,0.751109,0.084934
1,50,0.835083,0.062300
2,75,0.885162,0.051688
3,100,0.903843,0.037899
4,150,0.924591,0.027612
5,200,0.935970,0.024382
6,250,0.942671,0.023917
7,300,0.944810,0.019735
8,400,0.953649,0.015151
9,500,0.956211,0.014626


### 4G. Merge final results of the ClinicalTextMiner 

In [8]:
### Because the bootstrapped predictions on the final test required significant computational time,
### the results were computed in 4 batches based on the sample size and subsequently merged together.

import pandas as pd
import os

path_final_results = ### INSERT TARGET DIRECTORY ###

df_results_embed = pd.read_excel(os.path.join(path_final_results, 'part_1.xlsx'))
df_results_part_1 = pd.read_excel(os.path.join(path_final_results, 'part_1.xlsx'))
df_results_part_2 = pd.read_excel(os.path.join(path_final_results, 'part_2.xlsx'))
df_results_part_3 = pd.read_excel(os.path.join(path_final_results, 'part_3.xlsx'))
df_results_part_4 = pd.read_excel(os.path.join(path_final_results, 'part_4.xlsx'))

df_results_embed = df_results_part_1.append(df_results_part_2, ignore_index=True)
df_results_embed = df_results_embed.append(df_results_part_3, ignore_index=True)
df_results_embed = df_results_embed.append(df_results_part_4, ignore_index=True)

df_results_embed = df_results_embed[['sample_size', 'auc_mean', 'auc_std']]

df_results_embed.to_excel(os.path.join(path_final_results, 'final_results.xlsx'))
df_results_embed


,sample_size,auc_mean,auc_std
0,25,0.793640,0.052155
1,50,0.900822,0.038104
2,75,0.943463,0.022527
3,100,0.957711,0.016466
4,150,0.976549,0.008033
5,200,0.982854,0.004884
6,250,0.985870,0.002757
7,300,0.986877,0.001901
8,400,0.988390,0.001665
9,500,0.989212,0.001516
